# MC_Issues framework demo

In [1]:
from test_issues import IntIssue, StringIssue
from random import choice
from string import ascii_uppercase

## use metric_type.calculate_all(payload) get all issues found with a payload

In [2]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s)
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

FZDPSFHMTDEJQC is too long
FZDPSFHMTDEJQC contains duplicate letters
FZDPSFHMTDEJQC does not contain the letter 'a'
##########
DPDBGLH is too short
DPDBGLH contains duplicate letters
DPDBGLH does not contain the letter 'a'
##########
AFXLENJMSJLZYH is too long
AFXLENJMSJLZYH contains duplicate letters
##########
RGJIZDWH is too short
RGJIZDWH does not contain the letter 'a'
##########
MHUZWO is too short
MHUZWO does not contain the letter 'a'
##########
EDAFKUWAMWNXAJ is too long
EDAFKUWAMWNXAJ contains duplicate letters
##########
RHNHFKFBI is too short
RHNHFKFBI contains duplicate letters
RHNHFKFBI does not contain the letter 'a'
##########
ZQXDZJQOTOGM is too long
ZQXDZJQOTOGM contains duplicate letters
ZQXDZJQOTOGM does not contain the letter 'a'
##########
GJUDRNQAM is too short
##########
UFSKTWLNLFN is too long
UFSKTWLNLFN contains duplicate letters
UFSKTWLNLFN does not contain the letter 'a'
##########


## Use include_tags to only run on certain kinds of issue

In [3]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, include_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

BTLBTHUQMMWAD contains duplicate letters
##########
KMHSFQJVBILF contains duplicate letters
KMHSFQJVBILF does not contain the letter 'a'
##########
UDLDZ contains duplicate letters
UDLDZ does not contain the letter 'a'
##########
KLXOFFYDYHHR contains duplicate letters
KLXOFFYDYHHR does not contain the letter 'a'
##########
QDVDW contains duplicate letters
QDVDW does not contain the letter 'a'
##########
LUJQPMK does not contain the letter 'a'
##########
QMEZHFLQXJW contains duplicate letters
QMEZHFLQXJW does not contain the letter 'a'
##########
QXKMUCQSR contains duplicate letters
QXKMUCQSR does not contain the letter 'a'
##########
FJLFQERFGX contains duplicate letters
FJLFQERFGX does not contain the letter 'a'
##########
YPEYXLOYQ contains duplicate letters
YPEYXLOYQ does not contain the letter 'a'
##########


## And use exclude_tags to limit what kinds of issues you're checking for

In [4]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, exclude_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

DAXRY is too short
##########
ZAHIHJILC is too short
##########
No issues for ORVEQNXLLE
##########
No issues for CEJUWVCKEU
##########
ADISD is too short
##########
HUESAXSMA is too short
##########
IFJXELQKNZCF is too long
##########
VWOBVRHIZDGB is too long
##########
UVZBDNIHYKYQH is too long
##########
LLPJEPJTU is too short
##########


## The framework tries to catch and report errors in issue calculation gracefully
Note that execution isn't halted- we just log an error, and report that the issue didn't calculate right. 

In [7]:
print(IntIssue.calculate_all(2, exclude_tags=["breaking"]))
print(IntIssue.calculate_all(3))

ERROR:issues:Error calculating issue broken_issue for payload 3: Intentional Error
Traceback (most recent call last):
  File "/Users/pgulley/Projects/zammad_tests/issues.py", line 62, in calculate_all
    is_issue, result_data = issue_instance.calculate(payload)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pgulley/Projects/zammad_tests/test_issues.py", line 63, in calculate
    raise RuntimeError("Intentional Error")
RuntimeError: Intentional Error


[{'issue_name': 'is_even', 'template': '2 is even', 'tags': []}]
[{'issue_name': 'broken_issue', 'tags': ['breaking'], 'error': True, 'error_message': 'Intentional Error', 'template': "An error occurred while calculating 'broken_issue'"}]


# Usage:

We shouldn't need very many extensions on the base issue class- we'll certainly need a SourceIssue, FeedIssue, and a CollectionIssue but that's probably it. 
Regardless, generating one is a two-liner: 

In [12]:
from issues import IssueBase
from typing import TypeVar, Tuple, Generic, Type, Dict, Callable, List, Any

class FloatIssue(IssueBase[float]):
    _ISSUES: Dict[str, Type["FloatIssue"]] = {}


Then actually implimenting the issue cases requires creating a class with just two methods- `calculate(payload)` and `render_template()`, then decorating it correctly:

In [45]:
@FloatIssue.register("decimal_places", tags=['decimal'])
class DecimalPlaces(FloatIssue):
    def calculate(self, payload:float) -> Tuple[bool, Dict]:        
        """
        NB the return signature: Tuple[bool, Any]
        bool is ultimately interpreted as "is_issue" - if false, the result is skipped
        Dict is the input provided to the render template
        """
        num, dec = str(payload).split(".")
        exp = len(dec)
    
        return exp > 2, {'value':payload, "exp":exp}

    def render_template(self):
        return f"{self.result['value']} has too many decimal places ({self.result['exp']})"


@FloatIssue.register("negative")
class NegativeFloat(FloatIssue):
    def calculate(self, payload:float):
        return payload < 0, {'value':payload}

    def render_template(self):
        return f"{self.result['value']} is negative"
            

In [44]:
for i in (1.2, -4.7 ,1.24525):
    r = FloatIssue.calculate_all(i)
    print(r)

[]
[{'issue_name': 'negative', 'template': '-4.7 is negative', 'tags': []}]
[{'issue_name': 'decimal_places', 'template': '1.24525 has too many decimal places (5)', 'tags': ['decimal']}]


At dev time, you should only be calling an issue from the calculate_all method- there's no need to ever call issue.calculate or issue.render_template directly. 
